In [2]:
from collections import Counter

import pandas as pd
import statbotics

sb = statbotics.Statbotics()

In [13]:
all_matches = []
events_1 = sb.get_events(year=2022, week=1, fields=["key"])
events_2 = sb.get_events(year=2022, week=2, fields=["key"])
for week, events in enumerate([[], events_1, events_2]):
    for event in events:
        try:
            matches = sb.get_matches(event=event["key"])
            for match in matches:
                match["week"] = week
            all_matches.extend(matches)
        except Exception:
            pass
matches_df = pd.DataFrame(all_matches)
matches_df.head()

,key,year,event,comp_level,set_number,match_number,playoff,time,status,red,...,blue_1,red_2,blue_2,red_endgame,blue_endgame,red_fouls,blue_fouls,red_no_fouls,blue_no_fouls,week
0,2022bcvi_qm1,2022,2022bcvi,qm,1,1,False,1646426638,Completed,"7332,4421,7787",...,3.0,0.0,0.0,15.0,6.0,0.0,4.0,35.0,11.0,1
1,2022bcvi_qm2,2022,2022bcvi,qm,1,2,False,1646427312,Completed,"7287,7190,8338",...,18.0,0.0,0.0,0.0,4.0,0.0,0.0,7.0,26.0,1
2,2022bcvi_qm3,2022,2022bcvi,qm,1,3,False,1646428102,Completed,"216,7167,7173",...,13.0,0.0,0.0,0.0,6.0,0.0,0.0,2.0,25.0,1
3,2022bcvi_qm4,2022,2022bcvi,qm,1,4,False,1646428873,Completed,"6485,244,8082",...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,28.0,6.0,1
4,2022bcvi_qm5,2022,2022bcvi,qm,1,5,False,1646429430,Completed,"288,7167,216",...,14.0,0.0,0.0,6.0,0.0,0.0,0.0,23.0,20.0,1


In [19]:
out = []
for _, m in matches_df[(matches_df.red_no_fouls >= 100) | (matches_df.blue_no_fouls >= 100)].iterrows():
    for alliance in ["red", "blue"]:
        if m[alliance + "_no_fouls"] < 100:
            continue
            
        out.append({
            "teams": [int(x) for x in m[alliance].split(",")],
            "event": m.event,
            "week": m.week,
            "comp_level": m.comp_level,
            "set_number": m.set_number,
            "match_number": m.match_number,
            "alliance": alliance,
            "auto": int(m[alliance + "_auto"]),
            "teleop": int(m[alliance + "_teleop"]),
            "endgame": int(m[alliance + "_endgame"]),
            "score": int(m[alliance + "_no_fouls"]),
            "cargo_rp": bool(m[alliance + "_rp_1"]),
            "hanger_rp": bool(m[alliance + "_rp_2"]),
        })
        
matches_100_df = pd.DataFrame(out)
display(matches_100_df.sort_values(by=["event"], ascending=True))

teams = []
for t in matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 4], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
38,"[4774, 5584, 5985]",2022ausc,2,qm,1,55,blue,28,62,16,106,True,True
39,"[8521, 7157, 1339]",2022caoc,2,qm,1,8,red,18,76,27,121,True,True
40,"[4201, 987, 294]",2022caoc,2,qm,1,21,blue,34,62,10,106,True,False
41,"[987, 3476, 5805]",2022caoc,2,qm,1,38,red,28,42,30,100,True,True
42,"[987, 3476, 3952]",2022caoc,2,qf,1,2,red,30,44,30,104,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,"[5724, 2363, 3136]",2022va305,1,sf,1,1,red,26,66,16,108,False,False
36,"[4089, 2910, 4309]",2022wasno,1,sf,1,1,red,30,58,15,103,False,False
34,"[2910, 4205, 4089]",2022wasno,1,qm,1,14,blue,26,56,21,103,True,True
37,"[4089, 2910, 4309]",2022wasno,1,sf,1,2,red,30,64,15,109,False,False


[(4414, 10), (1678, 7), (987, 6), (589, 5), (1690, 5), (1577, 5), (3476, 5)]

In [15]:
qm_matches_100_df = matches_100_df[matches_100_df.comp_level == "qm"]
display(qm_matches_100_df)

teams = []
for t in qm_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 1], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
0,"[1678, 973, 5817]",2022caph,1,qm,1,2,red,20,81,27,128,True,True
1,"[4501, 6560, 4414]",2022caph,1,qm,1,15,blue,32,52,30,114,True,True
2,"[973, 5199, 6560]",2022caph,1,qm,1,31,red,30,58,16,104,True,True
3,"[1678, 3759, 3128]",2022caph,1,qm,1,38,blue,20,59,30,109,True,True
11,"[5554, 1690, 1577]",2022isde1,1,qm,1,2,red,22,62,21,105,True,True
16,"[2075, 3357, 7244]",2022mifor,1,qm,1,28,blue,30,52,21,103,True,True
24,"[5172, 7021, 3883]",2022mndu2,1,qm,1,78,blue,18,58,24,100,True,True
29,"[3310, 624, 5417]",2022txwac,1,qm,1,31,blue,22,60,21,103,True,True
30,"[148, 2848, 2687]",2022txwac,1,qm,1,50,red,26,58,24,108,True,True
34,"[2910, 4205, 4089]",2022wasno,1,qm,1,14,blue,26,56,21,103,True,True


[(1678, 2), (973, 2), (6560, 2), (987, 2)]

In [22]:
elim_matches_100_df = matches_100_df[matches_100_df.comp_level != "qm"]
display(elim_matches_100_df)

teams = []
for t in elim_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 3], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
4,"[4414, 1678, 589]",2022caph,1,qf,1,1,red,34,68,30,132,False,False
5,"[4414, 1678, 589]",2022caph,1,qf,1,2,red,34,86,16,136,False,False
6,"[4414, 1678, 589]",2022caph,1,sf,1,2,red,30,60,29,119,False,False
7,"[4414, 1678, 589]",2022caph,1,f,1,1,red,26,58,29,113,False,False
8,"[4414, 1678, 589]",2022caph,1,f,1,2,red,30,56,29,115,False,False
9,"[179, 4481, 4592]",2022flwp,1,sf,1,1,red,28,53,21,102,False,False
10,"[2168, 4206, 2641]",2022flwp,1,sf,2,1,red,28,46,27,101,False,False
12,"[1577, 1690, 6104]",2022isde1,1,qf,1,1,red,38,80,10,128,False,False
13,"[1577, 1690, 6104]",2022isde1,1,qf,1,2,red,30,101,15,146,False,False
14,"[1577, 1690, 6741]",2022isde1,1,f,1,1,red,36,74,0,110,False,False


[(4414, 9),
 (1678, 5),
 (589, 5),
 (1577, 4),
 (1690, 4),
 (987, 4),
 (3476, 4),
 (3952, 4),
 (359, 4),
 (5089, 4)]